<a href="https://colab.research.google.com/github/voodoohop/pollinations/blob/master/colabs/runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text to Image
### Method: CLIP + Siren
This model combines an image classifier called **CLIP** from *OpenAI* and an image generator called **SIREN.**

The process works like this:
1.  The **SIREN** model at first generates a completely random image
2.  The **CLIP** image classifier looks at the image and compares it with the given text caption
3.  Using this feedback the weights of the **SIREN** model are adjusted slightly to better approximate the image caption
4.  These steps are repeated until we stop it manually
---
In order to run click the [Colab] link. And then select *Runtime -> Run all* from the menu bar.

[colab]: https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb

## Parameters

In [3]:
text = 'An illustration of a black and white cat'  #@param {type: "string"}
args = '--lower_bound_cutout 0.5 --image-width=384 --iterations=1000 --epochs=2 --save-every=1 --save-progress=True --open_folder=False --overwrite=True --save_video=True --seed=123'  #@param {type: "string"}

parameters = {
    "text": text,
    "args": args
}

IPFS_PEER = "18.157.173.110"
ipfs_root = "/content/ipfs"

## Code (Frontend and IPFS Connection)

In [4]:
#@title Empty and create ipfs root: */content/ipfs*



!rm -r $ipfs_root
!mkdir -p $ipfs_root/input
!mkdir -p $ipfs_root/output
!echo "Created IPFS resource" > $ipfs_root/output/log

for key, default in parameters.items():
  print(default, file=open(f"{ipfs_root}/input/{key}", 'a'))



In [5]:
#@title Install IPFS-GO

!pip install aioipfs nest-asyncio

#!npm install -g nodemon


In [25]:
#@title Connect to local IPFS node and publish ipfs root
import sys
import asyncio
import aioipfs
import nest_asyncio
import json

#client = aioipfs.AsyncIPFS(debug=True)
clientRemote = aioipfs.AsyncIPFS(host=IPFS_PEER,port=5002,debug=True)
client=clientRemote
ipfs_nodeid = None
ipfs_contentid = None
async def get_node_id():
    return (await client.id())["ID"]

async def _add_and_publish(path=ipfs_root, pin_remote=False):
    global ipfs_contentid
    last_added=None
    async for added_file in client.add(path, recursive=True):
      print('Imported file', added_file)
      last_added=added_file["Hash"]
    ipfs_contentid = last_added
    update_frontend()
    return last_added

def add_and_publish():
  asyncio.run(loop.create_task(_add_and_publish(pin_remote=True)))

loop = asyncio.get_running_loop()
nest_asyncio.apply(loop)


ipfs_nodeid = asyncio.run(loop.create_task(get_node_id()))



#asyncio.run(loop.create_task(sub("hello")))



In [7]:
#@title Connect to IPFS Pubsub

async def subscribe(topic, handler=None):
  print("subscribed to",topic)
  async for message in client.pubsub.sub(topic):
    if handler:
      await handler(message)
    else:
      print("No handler. Received IPFS pub message",message)
    break

async def publish(topic, data):
  print("publishing",topic,data)
  await client.pubsub.pub(topic, data)

def on_frontend_data(handler):
  async def handle(ipfs_message):
    await handler(ipfs_message["data"])
  asyncio.run(loop.create_task(subscribe(ipfs_nodeid, handle)))

def update_frontend():
  asyncio.run(loop.create_task(publish(ipfs_nodeid, ipfs_contentid)))



add_and_publish()

In [4]:
#@title Create connection to frontend UI

import IPython
from google.colab import output

display(IPython.display.Javascript(f'''
  const iframe = document.createElement("iframe")
  iframe.src = `http://localhost:3000/incolab.html?node={ipfs_nodeid},{ipfs_contentid}`

  document.querySelector("#output-area").appendChild(iframe)
'''))



<IPython.core.display.Javascript object>

In [6]:
async def update_ipfs_content(contentid):
  global ipfs_contentid
  contentid = contentid.decode("utf-8")
  
  await client.get(contentid)
  !cp -rv $contentid/* $ipfs_root
  #!rm -r $contentid
  ipfs_contentid = contentid

on_frontend_data(update_ipfs_content)


In [5]:
from glob import glob
from os.path import basename
for path in glob(f"{ipfs_root}/input/*"):
  key = basename(path)
  with open(path,'r') as f:
    value = f.read()
    parameters[key] = value

print("New parameters", parameters)

New parameters {'text': 'An illustration of a black and white cat\n', 'args': '--lower_bound_cutout 0.5 --image-width=384 --iterations=1000 --epochs=2 --save-every=1 --save-progress=True --open_folder=False --overwrite=True --save_video=True --seed=123\n'}
subscribed to None


IPFSConnectionError: ignored

## CODE (DEEP-DAZE)

In [ ]:
!pip install deep-daze --upgrade
!pip install imageio-ffmpeg asyncio websockets varname --upgrade

In [ ]:
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import os
import json
import itertools
from base64 import b64encode
from time import sleep
import requests
import os
from mimetypes import guess_type 
import re
flatten = itertools.chain.from_iterable

GITHUB_REPO="voodoohop/pollinations/blob/master/colabs/deep-daze.ipynb"
GITHUB_REPO_CLEAN = re.sub('[^0-9a-zA-Z]+', '_', GITHUB_REPO)
print(GITHUB_REPO_CLEAN)
#parameters["text"] = "A black and white photo of Saeko with her new Elektron FM synthesizer"
def run(text, args, watch_media_paths=["./*.jpg","./*.mp4"]):
  print("Running",text)
  for to_remove in watch_media_paths:
    for filepath in glob(to_remove):
      print("Deleting",filepath,"before starting.")
      os.remove(filepath)

  arguments = args.split(" ")
  cmd = ['imagine'] + arguments +[text]
  
  p = Popen(cmd, stdout = PIPE, 
          stderr = STDOUT, shell = True)

  found_media_files=[]
  
  while True:
    line = p.stdout.readline()#
    if not line: break

    stdout.write(line)
    with open(f"{ipfs_root}/log","w") as log_out:
      log_out.write(line.decode("utf-8"))
      log_out.flush()
    #headers = { 
    #    "params": params,
    #    "text": text
    #}
    #conn.send(body=line.decode("utf-8"), headers=headers, destination='/topic/colabOut')
    #print("new contentid", client.add(ipfs_root, recursive=True))
    
    #continue
    new_found_media_files = get_matching_files(watch_media_paths)
    new_found_media_files.sort(key=os.path.getmtime)
    added_media_files = list(set(new_found_media_files)-set(found_media_files))
    found_media_files = new_found_media_files
    pin=False
    if len(added_media_files) > 0:
      for added in added_media_files:
        sleep(0.1)
        if len(added) > 0:
          !cp $added $ipfs_root/output
          print("Adding:", added,"to ipfs root")
          pin=True
    add_and_publish()    
       

def get_matching_files(search_paths):
  return list(flatten([glob(search_path) for search_path in search_paths]))

run(**parameters)

sleep(1000000)


In [ ]:
!pip install gdown
!gdown --id 1rUt0PiW3SDJ5SZnCsv1CU1CUBTC3wwZa
%run Deep_Daze.ipynb

# DISABLED
```
!wget wget https://dist.ipfs.io/go-ipfs/v0.8.0/go-ipfs_v0.8.0_linux-amd64.tar.gz
!tar -xvzf go-ipfs_v0.8.0_linux-amd64.tar.gz
%cd go-ipfs
!sudo bash install.sh
%cd -
```

# DISABLED
```
#@title Install IPFS-GO

!pip install aioipfs nest-asyncio

!wget wget https://dist.ipfs.io/go-ipfs/v0.8.0/go-ipfs_v0.8.0_linux-amd64.tar.gz
!tar -xvzf go-ipfs_v0.8.0_linux-amd64.tar.gz
%cd go-ipfs
!sudo bash install.sh
%cd -

#!npm install -g nodemon
```

```
#@title Start IPFS daemon and connect to public daemon
!killall ipfs 2>/dev/null
!sleep 2
!killall -9 ipfs 2>/dev/null
#!nohup jsipfs daemon --enable-namesys-pubsub &
!ipfs init
!ipfs config Addresses.Gateway /ip4/0.0.0.0/tcp/9001
!nohup ipfs daemon --enable-namesys-pubsub --enable-pubsub-experiment &
!sleep 12
#!ipfs config Addresses.API /ip4/127.0.0.1/tcp/5002
#!jsipfs config --json API.HTTPHeaders.Access-Control-Allow-Origin '["*"]'
#!jsipfs config --json API.HTTPHeaders.Access-Control-Allow-Methods '["PUT", "GET", "POST"]'
!tail nohup.out
!ipfs swarm connect /ip4/$IPFS_PEER/tcp/4002/p2p/QmWxrFJqurnyedzJ5rzpgycTqSikJ7kcy7eHmqYnndKhwr


# This is formatted as code
```

